In [3]:
from selenium import webdriver
from time import sleep
import time
from selenium.webdriver.common.by import By
import pandas as pd

In [4]:
data = pd.ExcelFile("./data/Ahrefページ価値取得.xlsx")
data

In [7]:
input_sheet_name = data.sheet_names
data = data.parse(input_sheet_name[0])
data

,ドメイン,トラフィック,価値,キーワード
0,mens-datsumo.jp,NaN,NaN,NaN
1,epinity.com,NaN,NaN,NaN
2,jfam.co.jp,NaN,NaN,NaN
3,chicken-gym.jp,NaN,NaN,NaN
4,zakzak.co.jp,NaN,NaN,NaN
5,getfit.jp,NaN,NaN,NaN
6,daily-tohoku.news,NaN,NaN,NaN


In [8]:
USERNAME = "media_general@joint.works"
PASSWORD = "Jointinc"

options = webdriver.chrome.options.Options()
options.add_argument("-headless")
options.add_argument("-no-sandbox")
profile_path = '/Users/suzukiharumasaru/Library/Application Support/Google/Chrome/Profile ahrefs'
options.add_argument('--user-data-dir=' + profile_path)
driver = webdriver.Chrome(options=options)

target_url = "https://app.ahrefs.com/user/login"
driver.get(target_url)
sleep(5)

username_input = driver.find_element(By.XPATH,'//*[@id="root"]/div/div/div/div[1]/div/div/div/div/form/div/div[1]/div/input')
print("とれた！")

username_input.send_keys(USERNAME)
sleep(1)

password_input = driver.find_element(By.XPATH,'//*[@id="root"]/div/div/div/div[1]/div/div/div/div/form/div/div[2]/div/div/input')
password_input.send_keys(PASSWORD)
sleep(1)

login_burron = driver.find_element(By.XPATH,'//*[@id="root"]/div/div/div/div[1]/div/div/div/div/form/div/button/div')
login_burron.submit()
sleep(5)

df = pd.DataFrame()

for domain in data['ドメイン'].to_list():
    ahrefs_url = "https://app.ahrefs.com/positions-explorer/top-subfolders/subdomains/jp/all/all/all/all/all/all/all/1/traffic_desc?target="+domain+"%2F"

    driver.get(ahrefs_url)
    sleep(5)

    df_tmp = driver.find_element(By.XPATH,'//*[@id="main_se_data_table"]')

    html = df_tmp.get_attribute('outerHTML')
    df_tmp = pd.read_html(html)

    df_tmp = df_tmp[0].head(1)

    df_tmp = df_tmp[['トラフィック', '価値', 'キーワード']]
    df_tmp.insert(0,'ドメイン',domain)
    df_tmp.insert(4,'ahrefs上位ページ取得URL',ahrefs_url)

    df = pd.concat([df, df_tmp], join='outer')
driver.quit()

とれた！


In [9]:
df.to_excel('./output/output.xlsx',
            index=False)

In [10]:
data_2 = pd.ExcelFile("./output/output.xlsx")
data_2